In [1]:
import numpy as np
from tensorflow.keras.models import *
from tensorflow.keras.layers import *

In [2]:
X_train, X_test, Y_train, Y_test = np.load('./news_data_max_17_size_9656.npy', allow_pickle=True)

print(X_train.shape)
print(X_test.shape)
print(Y_train.shape)
print(Y_test.shape)

(7226, 17)
(803, 17)
(7226, 7)
(803, 7)


In [5]:
# 모델 생성
# Embedding: 다차원 공간 상에 word를 배치해, 의미벡터 부여. 비슷한 위치 파악.
'''
같은 자리에 들어가서 말이 된다면 유사하다고 볼 수 있음
오늘 날씨는 축구/야구/데이트하기 좋다 (!)
오늘 날씨는 딸기/엄마/커피하기 좋다 (?)
유사한 단어는 공간 상에 비슷한 위치에 둘 수 있다
'''
# Conv1D: 문장 1D
# 벡터사이즈 큰 것으로 고를 것임

'''
"t" 다음에 올 수 있는 철자는? "t" 다음에 "e"가 온다면?
이처럼 이전의 예측 결과는 다음 예측 결과에 영향을 준다 = RNN
RNN의 문제: 반복이 많다. 따라서 미분을 하다보면 기울기 소실 문제가 발생한다.
레이어가 깊어질수록, input_length가 길어질수록 값이 작아져서
(0.5보다 작은 값에 계속 곱하기 때문에) 학습 결과가 좋지 않다.

RNN의 단점을 보완한 것이 LSTM: 레이어를 건너 뛰어 모델에 넘겨주는 것.
따라서 순서정보가 중요할 때는 주로 LSTM을 쓴다.

이때 GRU는 LSTM과 비슷하게 순서정보를 학습하는 모델
tanh함수: LSTM이나 GRU에 쓴다. 이건 정해진 것이다.
'''
# return_sequences: 30개 문장이 입력되면, 30개 문장이 출력되도록 (False하면 맨 마지막 거 하나만 나옴)
# GRU한 결과를 또 GRU에 넣을 거면 True로 해야 한다

# Dropout: 매우 촘촘하게 연결된 Dense레이어에서 랜덤하게 일부를 학습하지 않는다.(망각)
# 과적합을 방지하기 위함이다.

# 성능 개선을 위해 몇 가지 더 빼고 추가함(수정 여러번 해서 확인함)

model = Sequential()
model.add(Embedding(9656, 300, input_length=17)) # 300차원
model.add(Conv1D(filters=32, kernel_size = 5, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=1))

# 3층의 GRU
model.add(GRU(128, activation = 'tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(64, activation = 'tanh', return_sequences=True))
model.add(Dropout(0.3))
model.add(GRU(64, activation = 'tanh'))
model.add(Dropout(0.3))
model.add(Flatten())

# Dense 레이어; 다중분류이므로 softmax
model.add(Dense(128, activation='relu'))
model.add(Dense(7, activation='softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 17, 300)           2896800   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 17, 32)            48032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 17, 32)            0         
_________________________________________________________________
gru_3 (GRU)                  (None, 17, 128)           62208     
_________________________________________________________________
dropout_3 (Dropout)          (None, 17, 128)           0         
_________________________________________________________________
gru_4 (GRU)                  (None, 17, 64)            37248     
_________________________________________________________________
dropout_4 (Dropout)          (None, 17, 64)           

In [6]:
# 학습
# validation시 학습하지 않도록

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
fit_hist = model.fit(X_train, Y_train, batch_size=100, epochs = 6, validation_data=(X_test, Y_test))

Train on 7226 samples, validate on 803 samples
Epoch 1/6
7226/7226 [==============================] - 16s 2ms/sample - loss: 1.4858 - accuracy: 0.3896 - val_loss: 0.8632 - val_accuracy: 0.6849
Epoch 2/6
7226/7226 [==============================] - 7s 1ms/sample - loss: 0.6179 - accuracy: 0.7822 - val_loss: 0.5793 - val_accuracy: 0.8082
Epoch 3/6
7226/7226 [==============================] - 7s 1ms/sample - loss: 0.2541 - accuracy: 0.9231 - val_loss: 0.6762 - val_accuracy: 0.7970
Epoch 4/6
7226/7226 [==============================] - 7s 980us/sample - loss: 0.1411 - accuracy: 0.9572 - val_loss: 0.7280 - val_accuracy: 0.8107
Epoch 5/6
7226/7226 [==============================] - 7s 976us/sample - loss: 0.0897 - accuracy: 0.9751 - val_loss: 0.8879 - val_accuracy: 0.8057
Epoch 6/6
7226/7226 [==============================] - 7s 944us/sample - loss: 0.0620 - accuracy: 0.9839 - val_loss: 0.8785 - val_accuracy: 0.8107


In [7]:
model.save('./news_finance.h5')

In [8]:
learned_model = load_model('./news_finance.h5')